In [ ]:
from bomberman.Environment import Environnement
import torch
from DQN_Agent import BuffedDQNAgent
import time

from IPython.display import clear_output

class Trainer():
	def __init__(self, Agent : BuffedDQNAgent) -> None:
		self.training_agent = Agent(1)
		self.env_1 = Environnement(1)

		self.to_beat_agent = Agent(2)
		self.env_2 = Environnement(2)

	def reset(self):
		# Player 1
		state_1 = self.env_1.reset()

		# Player 2
		state_2 = self.env_2.reset()
		return state_1, state_2
		
	def compute_reward(self, old_state, next_state, game_over):
		# TODO
		winner = next_state.as_tuple()[2]
		
		if game_over: # if game is over and player is not the winner, return -1 else 1
			return -1 if not winner or winner != self.training_agent.player_num else 1
		
		return 0

	def train(self, steps=10, log_each=1, skip_frames=4, reset_agent_each=10, max_duration=50):
		state_1, state_2 = self.reset()
		current_skip = skip_frames
		
		i = 1

		# Use a game duration to force agent to finish the game fast
		# (and avoid starting agent which are random to never end playing)
		game_duration = 0

		while i <= steps:
			game_over = False

			data_point = {"obs": state_2} # need [obs, action, reward, next_obs, done]

			while current_skip > 0 and not game_over:
				with torch.no_grad(): # Don't compute grad for game playing
					time.sleep(0.01)
					
					# To_beat_agent turn
					tobeat_action = self.to_beat_agent.get_action(state_1)
					state_1 = self.env_1.do_action(5) # put bomb to test game reset TODO replace with tobeat_action
					# Then the training agent turn
					training_action = self.training_agent.get_action(state_2)
					state_2 = self.env_2.do_action(training_action)

					# if it is the first step of the observation
					if current_skip == skip_frames:
						data_point["action"] = training_action 

					current_skip -= 1

					if (state_2.winner is not None) or game_duration >= max_duration:
						game_over = True
			
			current_skip = skip_frames
			game_duration += 1

			data_point["next_obs"] = state_2
			data_point["reward"] = self.compute_reward(data_point["obs"], data_point["next_obs"], game_over)
			data_point["done"] = game_over

			if game_over:
				game_duration = 0
				print("game has been reseted and winner was : ", data_point["next_obs"].as_tuple()[2]) # only for debut TODO remove this line
				state_1, state_2 = self.reset()

			# TODO : add to replay buffer and train with replay buffer

			if i % reset_agent_each == 0: # reset the to beat agent to the current trained agent
				self.to_beat_agent.brain.load_state_dict(self.training_agent.brain.state_dict())

			if i % log_each == 0:
				print(f"{i} steps done")
			i += 1 # TODO : only update i when a step of training is done with the memory

In [ ]:
trainer = Trainer(BuffedDQNAgent)

In [ ]:
trainer.train(steps=1000, log_each=100, reset_agent_each=300, max_duration=10)